In [72]:
import itertools
import numpy as np
from numpy import linalg as LA
import math

In [73]:
# number of sites
N = 4

# constants
U = 1
t = 1

def generate_basis_states(n):
    """
    Generates basis states for half filling of N sites
    """
    result = []
    int_result = []
    for bits in itertools.combinations(range(n * 2), n):
        # only choosing N bits from N * 2 bits, so half filling
        s = ['0'] * n * 2
        for bit in bits:
            # first n bits are up, second n are down
            # sites go 0 to N - 1 for each spin
            s[bit] = '1'
        
        if (''.join(s)[:N].count('1') == N / 2): # half filling
            int_result.append(int(''.join(s),2)) # integer representation of bit basis
            result.append(''.join(s))            # bit (string) basis
    # order ascending
    return (result[::-1], int_result[::-1])

(result, basis_states) = generate_basis_states(N)

In [74]:
for i in range(len(result)):
    print('{0}\t\t|{1}>\t\t{2}'.format(i+1, result[i], basis_states[i]))

1		|00110011>		51
2		|00110101>		53
3		|00110110>		54
4		|00111001>		57
5		|00111010>		58
6		|00111100>		60
7		|01010011>		83
8		|01010101>		85
9		|01010110>		86
10		|01011001>		89
11		|01011010>		90
12		|01011100>		92
13		|01100011>		99
14		|01100101>		101
15		|01100110>		102
16		|01101001>		105
17		|01101010>		106
18		|01101100>		108
19		|10010011>		147
20		|10010101>		149
21		|10010110>		150
22		|10011001>		153
23		|10011010>		154
24		|10011100>		156
25		|10100011>		163
26		|10100101>		165
27		|10100110>		166
28		|10101001>		169
29		|10101010>		170
30		|10101100>		172
31		|11000011>		195
32		|11000101>		197
33		|11000110>		198
34		|11001001>		201
35		|11001010>		202
36		|11001100>		204


In [75]:
# setting up U term in half-filling space
h_u = np.array([[ 0 for i in range(len(result))] for j in range(len(result))])

# separate the bit basis into up (first N) and down (last N) spins
# implemented very complicated for some reason
# spins = [(bin((x >> N) & int('1'*N, 2))[2:].zfill(N), bin(x & int('1'*N, 2))[2:].zfill(N)) for x in basis_states]
#          up     down
spins = [(x[:N], x[N:]) for x in result]

for i in range(len(spins)):
    # if two particles are on the same site this is recorded in h_u. Only diagonal is affected
    # i.e. ('00', '11') --> 0*1 + 0*1 = 0
    #      ('01', '01') --> 0*0 + 1*1 = 1  both particles are on side 1
    s = sum([int(spins[i][0][j]) * int(spins[i][1][j]) for j in range(N)])
    if (s > 0):
        print(result[i] + ' is doubly occupied')
        h_u[i][i] = 1

# U_1 is state 0101 [0][0]  
# h_u[0][0] = 0

# multiply by U
h_u *= U
print()
print(h_u)

00110011 is doubly occupied
00110101 is doubly occupied
00110110 is doubly occupied
00111001 is doubly occupied
00111010 is doubly occupied
01010011 is doubly occupied
01010101 is doubly occupied
01010110 is doubly occupied
01011001 is doubly occupied
01011100 is doubly occupied
01100011 is doubly occupied
01100101 is doubly occupied
01100110 is doubly occupied
01101010 is doubly occupied
01101100 is doubly occupied
10010011 is doubly occupied
10010101 is doubly occupied
10011001 is doubly occupied
10011010 is doubly occupied
10011100 is doubly occupied
10100011 is doubly occupied
10100110 is doubly occupied
10101001 is doubly occupied
10101010 is doubly occupied
10101100 is doubly occupied
11000101 is doubly occupied
11000110 is doubly occupied
11001001 is doubly occupied
11001010 is doubly occupied
11001100 is doubly occupied

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [76]:
# epsilon term. Only apply when N = 2?
# this depends on the ordering of the sites
# |site 0 up, site 1 up, site 0 down, site 1 down>

e0 = 0
e1 = 0
if (N == 2 and e1 != 0):
    h_e0 = np.array([[ 0 for i in range(len(result))] for j in range(len(result))])
    h_e1 = np.array([[ 0 for i in range(len(result))] for j in range(len(result))])
    for i in range(len(result)):
        # count the number of particles on site one/zero
        for j in range(N):
            h_e0[i][i] += int(result[i][2*j])
            h_e1[i][i] += int(result[i][2*j+1])
    h_e0 *= e0
    h_e1 *= e1
    h_u += (h_e1 + h_e0)
    print(h_u)

In [77]:
def hop(state):
    def ns(s):
        """
        Finds the possible neighbor states for a state s. Spins can't flip while hopping
        """
        hops = []
        for i in range(len(s)):
            if(s[i] == '1'):
                temp = list(s)
                # see if it can hop one site forward (with periodic bounding conditions)
                # can only hop if final site is unoccupied
                if (i + 1 == len(s)):
                    # hopping from site n - 1 to 0
                    if (s[0] == '0'):
                        temp[i] = '0'
                        temp[0] = '1'
                        # counting number of particles to hop through
                        if ((-1)**temp[1:-1].count('1') == 1):
                            hops.append(''.join(temp))
                        else: 
                            hops.append('-' + ''.join(temp))
                        temp = list(s)
                else:
                    # hopping one site ahead
                    if (s[i + 1] == '0'):
                        temp[i] = '0'
                        temp[i + 1] = '1'
                        hops.append(''.join(temp))
                        temp = list(s)
                if (i - 1 < 0):
                    # hopping from site 0 to n - 1
                    if (s[-1] == '0'):
                        temp[i] = '0'
                        temp[-1] = '1'
                        # counting number of particles to hop through
                        if ((-1)**temp[1:-1].count('1') == 1):
                            hops.append(''.join(temp))
                        else: 
                            hops.append('-' + ''.join(temp))
                        temp = list(s)
                else:
                    # hopping one site backwards
                    if (s[i - 1] == '0'):
                        temp[i] = '0'
                        temp[i - 1] = '1'
                        hops.append(''.join(temp))
                        temp = list(s)
        return hops
    
    neighbor_states = []
    # find possible hops for particles with up spin
    up_hops = list(dict.fromkeys(ns(state[0])))
    # find possible hops for particles with down spin
    down_hops = list(dict.fromkeys(ns(state[1])))

    for i in range(len(down_hops)):
        if (down_hops[i][0] == '-'):
            hop = basis_states.index(int(state[0] + down_hops[i][1:], 2))
            if (hop == 0):
                neighbor_states.append(-0.0)
            else:
                neighbor_states.append(-1 * hop)
        else:
            neighbor_states.append(basis_states.index(int(state[0] + down_hops[i], 2)))
    for i in range(len(up_hops)):
        if (up_hops[i][0] == '-'):
            hop = basis_states.index(int(up_hops[i][1:] + state[1], 2))
            if (hop == 0):
                neighbor_states.append(-0.0)
            else:
                neighbor_states.append(-1 * hop)
        else:
            neighbor_states.append(basis_states.index(int(up_hops[i] + state[1], 2)))
    
    return neighbor_states

# setting up t term in half-filling space
h_t = np.array([[ 0 for i in range(len(result))] for j in range(len(result))])


for i in range(len(spins)):
    ns = hop(spins[i])
    print('state', i, '(', result[i], ') neighbor states', ns)
    for j in range(len(ns)):
        if (math.copysign(1, ns[j]) == -1.0):
            h_t[i][abs(int(ns[j]))] = -1
        else:
            h_t[i][ns[j]] = 1
# multiply by -t
h_t *= -t
print()
print(h_t)


state 0 ( 00110011 ) neighbor states [1, -4, 6, -24]
state 1 ( 00110101 ) neighbor states [0, 3, -5, 2, 7, -25]
state 2 ( 00110110 ) neighbor states [4, 1, 8, -26]
state 3 ( 00111001 ) neighbor states [1, 4, 9, -27]
state 4 ( 00111010 ) neighbor states [2, -0.0, 3, 5, 10, -28]
state 5 ( 00111100 ) neighbor states [-1, 4, 11, -29]
state 6 ( 01010011 ) neighbor states [7, -10, 0, 18, -30, 12]
state 7 ( 01010101 ) neighbor states [6, 9, -11, 8, 1, 19, -31, 13]
state 8 ( 01010110 ) neighbor states [10, 7, 2, 20, -32, 14]
state 9 ( 01011001 ) neighbor states [7, 10, 3, 21, -33, 15]
state 10 ( 01011010 ) neighbor states [8, -6, 9, 11, 4, 22, -34, 16]
state 11 ( 01011100 ) neighbor states [-7, 10, 5, 23, -35, 17]
state 12 ( 01100011 ) neighbor states [13, -16, 24, 6]
state 13 ( 01100101 ) neighbor states [12, 15, -17, 14, 25, 7]
state 14 ( 01100110 ) neighbor states [16, 13, 26, 8]
state 15 ( 01101001 ) neighbor states [13, 16, 27, 9]
state 16 ( 01101010 ) neighbor states [14, -12, 15, 17, 28

In [78]:
h = h_u + h_t
np.savetxt('h.txt', h, '%-2d', ', ')
print(h)

[[ 0 -1  0 ...  0  0  0]
 [-1  0 -1 ...  0  0  0]
 [ 0 -1  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0 -1  0]
 [ 0  0  0 ... -1  0 -1]
 [ 0  0  0 ...  0 -1  0]]


In [79]:
w, v = LA.eigh(h)
print(w)

[-4.00000000e+00 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00
 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00
 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00
 -1.74513415e-15 -1.50971693e-15 -1.09092132e-15 -9.97063260e-16
 -7.80418332e-16 -2.99232392e-16  8.38451404e-17  2.64453737e-16
  6.82580306e-16  1.31204199e-15  1.47906911e-15  2.37558133e-15
  2.00000000e+00  2.00000000e+00  2.00000000e+00  2.00000000e+00
  2.00000000e+00  2.00000000e+00  2.00000000e+00  2.00000000e+00
  4.00000000e+00  4.00000000e+00  4.00000000e+00  4.00000000e+00]


In [80]:
gsEigen = np.array(result)[abs(v[:,0]) > 0]
hf = []
for i in range(len(gsEigen)):
    if (gsEigen[i][:N] == gsEigen[i][N:]):
        hf.append(gsEigen[i])
        
print(len(hf))
print(hf)

6
['00110011', '01010101', '01100110', '10011001', '10101010', '11001100']


In [81]:
v = np.zeros(36)
v[1] = 1
np.dot(h, v)

array([-1.,  0., -1., -1.,  0.,  1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [82]:
def creation(state, site, spin):
    """
    Creates a particle at site 'site' with spin 'spin', if possible
    |0011> has sites 0, 1, in that order per N sites
    param spin: 1 - up, 0 - down
    """
    state = list(state)
    #             up                    down
    particle = state[:N] if spin else state[N:]
    if (particle[site] == '1'):
        # Can't add a particle to an already filled site, return vacuum state
        return ''
    else:
        particle[site] = '1'
        if (spin):
            state[:N] = particle
        else:
            state[N:] = particle
        return state

def annihilation(state, site, spin):
    """
    Annihilates a particle at site 'site' with spin 'spin', if possible
    |0011> has sites 0, 1, in that order per N sites
    param spin: 1 - up, 0 - down
    """
    state = list(state)
    #             up                    down
    particle = state[:N] if spin else state[N:]

    if (particle[site] == '0'):
        # State has been destroyed, so return vacuum state
        return ''
    else:
        particle[site] = '0'
        if (spin):
            state[:N] = particle
        else:
            state[N:] = particle
        return state


In [83]:
# Building the single particle density matrix
# 1 for up, 0 for down
# sigma1, sigma2 up to N. i, j sites

spdm = {}

for sigma1 in range(2):
    for i in range(N):        
        for sigma2 in range(2):
            for j in range(N):
                # setting up M(i, j, sigma1, sigma2)
                arr = np.array([[ 0 for i in range(len(result))] for j in range(len(result))])
                
                for mu in range(len(result)):
                    for nu in range(len(result)):
                        # < nu | a_{i, sigma1}^dagger a_{j, sigma2} | mu >
                
                        phi1 = annihilation(result[nu], i, sigma1)
                        phi2 = annihilation(result[mu], j, sigma2)
                        if ((phi1 != '' and phi2 != '') and phi1 == phi2):
                            arr[mu][nu] = 1
                spdm[str(i)+str(sigma1)+str(j)+str(sigma2)] = arr

In [84]:
almost_spdm_arr = np.array([0.0 for i in range(len(spdm))])
for i, j in enumerate(spdm.values()):
    almost_spdm_arr[i] = np.dot(v[:, 0].conj().T, np.dot(j, v[:, 0]))
asa = np.reshape(almost_spdm_arr, (2*N,2*N))
print(np.around(asa, decimals=2))
print(np.trace(asa))

IndexError: too many indices for array